<a href="https://colab.research.google.com/github/BlueSky2311/NAMD/blob/main/namd_Final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Hướng dẫn:
!!!! Luôn chỉnh đường dẫn file mỗi khi sử dụng
- Lưu file zip vào GDrive
- Extract từ file zip từ GDrive (sau khi import GDrive trỏ đường dẫn file zip trong drive vào địa chỉ ở cell 2)
- Nếu là lần đầu thì nên copy thư mục vừa giải nén vào GDrive
- Sửa đường dẫn file input và ouput ở cell 4
- Chạy NAMD

* Trường hợp disconnect giữa chừng thì reconnect lại và copy các file cần thiết để chạy lại: *.log *.coor *.vel *.xsc *.xst *.dcd
Mở file .xst lên xem đang chạy tới bao nhiêu, thêm dòng Firtsttimestep với giá trị = số step đang chạy mà bị dừng
Mở file .inp: sửa tên file bin, output và numstep/run

In [ ]:
#@title Load google drive
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
#@title Extract CHARMM-GUI files
# !tar -zxf /content/drive/MyDrive/namd/charmm-gui.tgz

In [ ]:
#@title Install NAMD
# Download the NAMD
!wget https://www.ks.uiuc.edu/Research/namd/3.0b5/download/019723/NAMD_3.0b5_Linux-x86_64-multicore-CUDA.tar.gz
#Extract NAMD
!tar -zxf /content/NAMD_3.0b5_Linux-x86_64-multicore-CUDA.tar.gz

In [ ]:
#@title Run NAMD/chạy NAMD
#Change the paths to correspond to the files/Đổi đường dẫn, ví dụ B1 -> B2 tùy vào cấu trúc muốn chạy (có thể xài Ctrl + H đổi nhanh)
!/content/NAMD_3.0b4_Linux-x86_64-multicore-CUDA.tar.gz +auto-provision +idlepoll +devices 0 /content/drive/MyDrive/namd/B1/B1/namd/step4_equilibration.inp > /content/drive/MyDrive/namd/B1/B1/namd/step4_equilibration.log &

In [ ]:
#@title Copy file to GDrive
#ls /content/drive/MyDrive/namd/B1/B1/namd
!cp *.log *.coor *.vel *.xsc *.xst *.dcd /content/drive/MyDrive

To combine multiples .dcd files, see this notebook:https://colab.research.google.com/github/BlueSky2311/NAMD/blob/main/CatDCD.ipynb

References:

1. https://www.youtube.com/watch?v=NHb5s9zDZP8
2. James C. Phillips, David J. Hardy, Julio D. C. Maia, John E. Stone, João V. Ribeiro, Rafael C. Bernardi, Ronak Buch, Giacomo Fiorin, Jérôme Hénin, Wei Jiang, Ryan McGreevy, Marcelo C. R. Melo, Brian Radak, Robert D. Skeel, Abhishek Singharoy, Yi Wang, Benoît Roux, Aleksei Aksimentiev, Zaida Luthey-Schulten, Laxmikant V. Kalé, Klaus Schulten, Christophe Chipot, and Emad Tajkhorshid. Scalable molecular dynamics on CPU and GPU architectures with NAMD. Journal of Chemical Physics, 153:044130, 2020. (PMC: PMC7395834)

